In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, Reshape, Input, concatenate, Layer, Activation, Flatten,
    Dropout, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from skimage.metrics import structural_similarity as ssim
import numpy as np
from sklearn.metrics import average_precision_score

from tensorflow.keras.layers import LayerNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd "/content/drive/MyDrive/MKA/semester 2/CVL/Tugas Kelompok/code"

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("reubensuju/celeb-df-v2")

# print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, Input, Layer, GlobalAveragePooling1D, LayerNormalization
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras.models import Model

In [ ]:
# ----------------------
# VisionTemporalTransformer (simplified, stable)
# ----------------------
class VisionTemporalTransformer(Layer):
    def __init__(self, patch_size=8, d_model=128, num_heads=4, spatial_layers=1, temporal_layers=1, **kwargs):
        super(VisionTemporalTransformer, self).__init__(**kwargs)
        self.patch_size = patch_size
        self.d_model = d_model
        self.num_heads = num_heads
        self.spatial_layers = spatial_layers
        self.temporal_layers = temporal_layers

        # basic projection for patches
        self.dense_projection = Dense(d_model)
        self.pos_emb = None

        # build small stacks of MHA + FFN for spatial and temporal
        self.spatial_mhas = [tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads) for _ in range(spatial_layers)]
        self.spatial_norm1 = [LayerNormalization() for _ in range(spatial_layers)]
        self.spatial_ffn = [tf.keras.Sequential([Dense(d_model*4, activation='relu'), Dense(d_model)]) for _ in range(spatial_layers)]
        self.spatial_norm2 = [LayerNormalization() for _ in range(spatial_layers)]

        self.temporal_mhas = [tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads) for _ in range(temporal_layers)]
        self.temporal_norm1 = [LayerNormalization() for _ in range(temporal_layers)]
        self.temporal_ffn = [tf.keras.Sequential([Dense(d_model*4, activation='relu'), Dense(d_model)]) for _ in range(temporal_layers)]
        self.temporal_norm2 = [LayerNormalization() for _ in range(temporal_layers)]

    def build(self, input_shape):
        # input_shape: (batch, frames, H, W, C)
        _, frames, H, W, C = input_shape
        ph = H // self.patch_size
        pw = W // self.patch_size
        num_patches = ph * pw
        # positional embedding for patches
        self.pos_emb = self.add_weight(shape=(1, num_patches, self.d_model), initializer='random_normal', trainable=True, name='pos_emb')
        super(VisionTemporalTransformer, self).build(input_shape)

    def call(self, inputs):
        # inputs shape: (batch, frames, H, W, C)
        batch = tf.shape(inputs)[0]
        frames = tf.shape(inputs)[1]
        H = tf.shape(inputs)[2]
        W = tf.shape(inputs)[3]
        C = tf.shape(inputs)[4]

        # reshape to (batch*frames, H, W, C)
        reshaped = tf.reshape(inputs, (-1, H, W, C))

        # extract patches
        patches = tf.image.extract_patches(
            images=reshaped,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1,1,1,1],
            padding='VALID'
        )
        # patches shape: (batch*frames, ph, pw, patch_dim)
        patch_dim = tf.shape(patches)[-1]
        num_patches = tf.shape(patches)[1] * tf.shape(patches)[2]
        patches = tf.reshape(patches, (-1, num_patches, patch_dim))

        # linear projection
        x = self.dense_projection(patches) + self.pos_emb

        # spatial transformer (per frame)
        for i in range(self.spatial_layers):
            attn = self.spatial_mhas[i](query=x, value=x, key=x)
            x = self.spatial_norm1[i](x + attn)
            ff = self.spatial_ffn[i](x)
            x = self.spatial_norm2[i](x + ff)

        # reshape back to (batch, frames, num_patches, d_model)
        d_model = tf.shape(x)[-1]
        x = tf.reshape(x, (batch, frames, -1, d_model))

        # temporal pooling across patches -> sequence of frame tokens
        x = tf.reduce_mean(x, axis=2)  # (batch, frames, d_model)

        # temporal transformer
        for i in range(self.temporal_layers):
            attn = self.temporal_mhas[i](query=x, value=x, key=x)
            x = self.temporal_norm1[i](x + attn)
            ff = self.temporal_ffn[i](x)
            x = self.temporal_norm2[i](x + ff)

        # pooled output per sample
        pooled = GlobalAveragePooling1D()(x)  # (batch, d_model)
        return pooled

# ----------------------
# Consistency loss based on cosine similarity
# ----------------------

def batch_consistency_loss(y_true, features):
    """
    features: (batch, dim)
    compute for each sample i: loss_i = 1 - mean_j cosine_sim(f_i, f_j)
    return tensor shape (batch,)
    """
    # flatten
    f = tf.reshape(features, (tf.shape(features)[0], -1))
    # L2 normalize
    f_norm = tf.math.l2_normalize(f, axis=1)
    sim_matrix = tf.matmul(f_norm, f_norm, transpose_b=True)  # (B, B)
    avg_sim = tf.reduce_mean(sim_matrix, axis=1)  # (B,)
    loss_per_sample = 1.0 - avg_sim
    return loss_per_sample

# wrapper to fit signature (y_true, y_pred) -> loss
def consistency_loss_wrapper(y_true, y_pred):
    return batch_consistency_loss(y_true, y_pred)

# ----------------------
# Build full model (two outputs)
# ----------------------
def build_lipinc_model(frame_shape=(8,64,144,3), residue_shape=(7,64,144,3), d_model=128):

    frame_input = Input(shape=frame_shape, name='FrameInput')
    residue_input = Input(shape=residue_shape, name='ResidueInput')

    # Vision Temporal Transformer
    vt = VisionTemporalTransformer(
        patch_size=8,
        d_model=d_model,
        num_heads=4,
        spatial_layers=1,
        temporal_layers=1
    )

    frame_feat = vt(frame_input)      # (batch, d_model)
    residue_feat = vt(residue_input)  # (batch, d_model)

    # -------------------------
    # FIXED: wrap tf.expand_dims
    # -------------------------
    expand1 = Lambda(lambda x: tf.expand_dims(x, axis=1))
    q = expand1(frame_feat)
    k = expand1(residue_feat)
    v = k

    # MultiHeadAttention
    mha = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=d_model//4)
    attn_out = mha(query=q, value=v, key=k)  # (batch, 1, d_model)

    # -------------------------
    # FIXED: wrap tf.squeeze
    # -------------------------
    squeeze = Lambda(lambda x: tf.squeeze(x, axis=1))
    attn_out = squeeze(attn_out)

    # -------------------------
    # FIXED: wrap tf.concat
    # -------------------------
    concat = Lambda(lambda t: tf.concat(t, axis=1))
    fusion = concat([frame_feat, residue_feat, attn_out])

    # Fusion MLP
    x = Dense(512, activation='relu')(fusion)
    x = Dense(256, activation='relu')(x)

    # Classification head
    class_output = Dense(2, activation='softmax', name='class_output')(x)

    # Features for consistency loss
    features_output = Dense(d_model, activation=None, name='features_output')(x)

    model = Model(
        inputs=[frame_input, residue_input],
        outputs=[class_output, features_output],
        name='LIPINC_fixed'
    )

    return model

# ----------------------
# Synthetic data generator (for demo)
# ----------------------

def generate_synthetic_dataset(num_samples=256, frame_shape=(8,64,144,3), residue_shape=(7,64,144,3), num_classes=2):
    X_frames = np.random.rand(num_samples, *frame_shape).astype(np.float32)
    X_residue = np.random.rand(num_samples, *residue_shape).astype(np.float32)
    y = np.random.randint(0, num_classes, size=(num_samples,))
    y_onehot = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    return (X_frames, X_residue), y_onehot

In [ ]:
from glob import glob
import cv2
import numpy as np
import kagglehub

# Download dataset
path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
print("Path to dataset files:", path)

DATASET_ROOT = path

def load_video(filepath, frame_count=8):
    cap = cv2.VideoCapture(filepath)
    frames = []
    while len(frames) < frame_count:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (144, 64))
        frames.append(frame)
    cap.release()
    if len(frames) < frame_count:
        return None
    return np.array(frames) / 255.0

def compute_residue(frames):
    residues = []
    for i in range(1, len(frames)):
        residues.append(frames[i] - frames[i-1])
    return np.array(residues)

real_videos = glob(f"{DATASET_ROOT}/Celeb-real/*.mp4")
fake_videos = glob(f"{DATASET_ROOT}/Celeb-synthesis/*.mp4")

X_frames, X_residues, Y = [], [], []

for vid in real_videos[:200]:
    frames = load_video(vid)
    if frames is None:
        continue
    residues = compute_residue(frames)
    X_frames.append(frames)
    X_residues.append(residues)
    Y.append([1, 0])

for vid in fake_videos[:200]:
    frames = load_video(vid)
    if frames is None:
        continue
    residues = compute_residue(frames)
    X_frames.append(frames)
    X_residues.append(residues)
    Y.append([0, 1])

X_frames = np.array(X_frames)
X_residues = np.array(X_residues)
Y = np.array(Y)

print("Dataset Loaded:", X_frames.shape, X_residues.shape)

Using Colab cache for faster access to the 'celeb-df-v2' dataset.
Path to dataset files: /kaggle/input/celeb-df-v2
Dataset Loaded: (400, 8, 64, 144, 3) (400, 7, 64, 144, 3)


In [ ]:
from sklearn.model_selection import train_test_split

# 1) Split Train + Temp (Val + Test)
Xf_train, Xf_temp, Xr_train, Xr_temp, y_train, y_temp = train_test_split(
    X_frames, X_residues, Y,
    test_size=0.30,    # 30% untuk val+test
    random_state=42,
    shuffle=True,
    stratify=Y
)

# 2) Split Temp menjadi Validation + Test
Xf_val, Xf_test, Xr_val, Xr_test, y_val, y_test = train_test_split(
    Xf_temp, Xr_temp, y_temp,
    test_size=0.50,   # 15% val + 15% test
    random_state=42,
    shuffle=True,
    stratify=y_temp
)

print("Train :", Xf_train.shape, Xr_train.shape, y_train.shape)
print("Val   :", Xf_val.shape,   Xr_val.shape,   y_val.shape)
print("Test  :", Xf_test.shape,  Xr_test.shape,  y_test.shape)

Train : (280, 8, 64, 144, 3) (280, 7, 64, 144, 3) (280, 2)
Val   : (60, 8, 64, 144, 3) (60, 7, 64, 144, 3) (60, 2)
Test  : (60, 8, 64, 144, 3) (60, 7, 64, 144, 3) (60, 2)


In [ ]:
# ------------------------------------------------------------
# 1. Build model
# ------------------------------------------------------------
model = build_lipinc_model()
model.summary()

# Optimizer
opt = Adam(learning_rate=1e-3)

# Compile model with 2 losses
model.compile(
    optimizer=opt,
    loss={
        'class_output': 'categorical_crossentropy',
        'features_output': consistency_loss_wrapper
    },
    loss_weights={
        'class_output': 1.0,
        'features_output': 5.0
    },
    metrics={'class_output': 'accuracy'}
)


# ------------------------------------------------------------
# 3. Buat dummy target untuk output kedua (features_output)
# ------------------------------------------------------------
dummy_train = np.zeros((len(y_train), 128), dtype=np.float32)
dummy_val   = np.zeros((len(y_val),   128), dtype=np.float32)
dummy_test  = np.zeros((len(y_test),  128), dtype=np.float32)


# ------------------------------------------------------------
# 4. TRAIN MODEL
# ------------------------------------------------------------
history = model.fit(
    x=[Xf_train, Xr_train],
    y={
        'class_output': y_train,
        'features_output': dummy_train
    },
    validation_data=(
        [Xf_val, Xr_val],
        {
            'class_output': y_val,
            'features_output': dummy_val
        }
    ),
    epochs=50,
    batch_size=16,
    shuffle=True
)


# ------------------------------------------------------------
# 5. EVALUATE MODEL
# ------------------------------------------------------------
results = model.evaluate(
    x=[Xf_test, Xr_test],
    y={
        'class_output': y_test,
        'features_output': dummy_test
    }
)

print("\nTest Results:")
print(results)


# ------------------------------------------------------------
# 6. SAVE MODEL
# ------------------------------------------------------------
model.save('lipinc_celebdf2_trained.h5')
print("Model saved to lipinc_celebdf2_trained.h5")

Model: "LIPINC_fixed"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ FrameInput          │ (None, 8, 64,     │          0 │ -                 │
│ (InputLayer)        │ 144, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ResidueInput        │ (None, 7, 64,     │          0 │ -                 │
│ (InputLayer)        │ 144, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vision_temporal_tr… │ (None, 128)       │    439,680 │ FrameInput[0][0], │
│ (VisionTemporalTra… │                   │            │ ResidueInput[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 128)    │          0 │ vision_temporal_… │
│                     │                   │            │ vision_temporal_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 128)    │     66,048 │ lambda[1][0],     │
│ (MultiHeadAttentio… │                   │            │ lambda[0][0],     │
│                     │                   │            │ lambda[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 128)       │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 384)       │          0 │ vision_temporal_… │
│                     │                   │            │ vision_temporal_… │
│                     │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 512)       │    197,120 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │    131,328 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_output        │ (None, 2)         │        514 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ features_output     │ (None, 128)       │     32,896 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 867,586 (3.31 MB)

 Trainable params: 867,586 (3.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - class_output_accuracy: 0.4413 - class_output_loss: 1.1164 - features_output_loss: 0.0043 - loss: 1.1384 - val_class_output_accuracy: 0.5000 - val_class_output_loss: 0.6940 - val_features_output_loss: 8.1603e-05 - val_loss: 0.6950
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - class_output_accuracy: 0.4504 - class_output_loss: 0.7204 - features_output_loss: 4.2142e-05 - loss: 0.7205 - val_class_output_accuracy: 0.5000 - val_class_output_loss: 0.7576 - val_features_output_loss: 2.7440e-05 - val_loss: 0.7617
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - class_output_accuracy: 0.5093 - class_output_loss: 0.7122 - features_output_loss: 1.4728e-05 - loss: 0.7121 - val_class_output_accuracy: 0.5000 - val_class_output_loss: 0.6955 - val_features_output_loss: 2.1712e-05 - val_loss: 0.6949
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - class_output_accuracy: 0.4157 - class_output_loss: 0.7073 - features_output_loss: 2.1220e-0


Test Results:
[0.6931575536727905, 0.6931772232055664, 5.960464477539063e-08, 0.5]
Model saved to lipinc_celebdf2_trained.h5


In [ ]:
import cv2
import numpy as np

# ============================================================
# FUNGSI PREPROCESSING VIDEO
# ============================================================

def load_video_for_test(video_path, frame_count=8, target_size=(144, 64)):
    """
    Load video dan ekstrak frame untuk testing

    Args:
        video_path: Path ke file video
        frame_count: Jumlah frame yang akan diambil (default: 8)
        target_size: (width, height) untuk resize frame (default: 144, 64)

    Returns:
        frames: numpy array shape (frame_count, height, width, 3)
        residues: numpy array shape (frame_count-1, height, width, 3)
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"❌ Cannot open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps if fps > 0 else 0

    print(f"\n📹 Video Information:")
    print(f"   File: {video_path}")
    print(f"   Total frames: {total_frames}")
    print(f"   FPS: {fps:.2f}")
    print(f"   Duration: {duration:.2f} seconds")
    print(f"   Resolution: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")

    frames = []
    # Sample frames evenly across the video (avoid last frame to prevent read errors)
    frame_indices = np.linspace(0, max(0, total_frames - 2), frame_count, dtype=int)

    print(f"\n🎬 Extracting {frame_count} frames at indices: {frame_indices}")

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()

        if not ret:
            print(f"⚠️  Warning: Cannot read frame at index {idx}, trying sequential read...")
            # Fallback: try reading sequentially
            cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, idx - 1))
            ret, frame = cap.read()
            if not ret:
                print(f"⚠️  Skipping frame at index {idx}")
                continue

        # Resize frame to model input size
        frame = cv2.resize(frame, target_size)
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    # If we got less frames than needed, pad with duplicate of last frame
    if len(frames) < frame_count:
        print(f"⚠️  Only got {len(frames)} frames, padding to {frame_count} frames...")
        while len(frames) < frame_count:
            frames.append(frames[-1].copy())  # duplicate last frame

    # Normalize frames to [0, 1]
    frames = np.array(frames, dtype=np.float32) / 255.0

    # Compute residues (frame differences)
    residues = []
    for i in range(1, len(frames)):
        residue = frames[i] - frames[i-1]
        residues.append(residue)

    residues = np.array(residues, dtype=np.float32)

    print(f"✅ Preprocessing complete!")
    print(f"   Frames shape: {frames.shape}")
    print(f"   Residues shape: {residues.shape}")

    return frames, residues


# ============================================================
# TEST VIDEO
# ============================================================

print("\n" + "="*70)
print(" " * 20 + "TESTING VIDEO: video.mp4")
print("="*70)

# Load and preprocess video
video_path = "video.mp4"

try:
    frames, residues = load_video_for_test(video_path)

    # Add batch dimension
    frames_batch = np.expand_dims(frames, axis=0)      # (1, 8, 64, 144, 3)
    residues_batch = np.expand_dims(residues, axis=0)  # (1, 7, 64, 144, 3)

    print(f"\n🔮 Running prediction...")

    # Predict using the trained model
    predictions = model.predict([frames_batch, residues_batch], verbose=0)
    class_output = predictions[0]  # (1, 2) - [prob_real, prob_fake]
    features_output = predictions[1]  # (1, 128)

    # Extract probabilities
    prob_real = class_output[0, 0]
    prob_fake = class_output[0, 1]

    # Determine label (threshold = 0.5)
    threshold = 0.5
    predicted_label = "FAKE" if prob_fake > threshold else "REAL"
    confidence = max(prob_real, prob_fake) * 100

    # Print results
    print("\n" + "="*70)
    print(" " * 25 + "🎯 PREDICTION RESULTS")
    print("="*70)
    print(f"\n{'Metric':<30s} {'Value':>15s}")
    print("-"*70)
    print(f"{'Real Probability':<30s} {prob_real:>10.4f} ({prob_real*100:>6.2f}%)")
    print(f"{'Fake Probability':<30s} {prob_fake:>10.4f} ({prob_fake*100:>6.2f}%)")
    print("-"*70)
    print(f"{'Predicted Label':<30s} {predicted_label:>15s}")
    print(f"{'Confidence':<30s} {confidence:>14.2f}%")
    print(f"{'Threshold Used':<30s} {threshold:>15.2f}")
    print("="*70)

    # Visual indicator
    if predicted_label == "FAKE":
        print("\n⚠️  WARNING: This video is predicted to be FAKE/DEEPFAKE!")
    else:
        print("\n✅ This video is predicted to be REAL/AUTHENTIC.")

    print("\n" + "="*70 + "\n")

    # Store results for later analysis
    video_result = {
        'video_path': video_path,
        'predicted_label': predicted_label,
        'prob_real': float(prob_real),
        'prob_fake': float(prob_fake),
        'confidence': float(confidence),
        'threshold': threshold
    }

    print("📊 Result Summary:")
    for key, value in video_result.items():
        print(f"   {key}: {value}")

except FileNotFoundError:
    print(f"\n❌ Error: Video file '{video_path}' not found!")
    print("   Please make sure the video file exists in the current directory.")

except Exception as e:
    print(f"\n❌ Error occurred while processing video:")
    print(f"   {type(e).__name__}: {str(e)}")
    import traceback
    traceback.print_exc()


                    TESTING VIDEO: video.mp4

📹 Video Information:
   File: video.mp4
   Total frames: 332
   FPS: 30.01
   Duration: 11.06 seconds
   Resolution: 576x1024

🎬 Extracting 8 frames at indices: [  0  47  94 141 188 235 282 330]
✅ Preprocessing complete!
   Frames shape: (8, 64, 144, 3)
   Residues shape: (7, 64, 144, 3)

🔮 Running prediction...

                         🎯 PREDICTION RESULTS

Metric                                   Value
----------------------------------------------------------------------
Real Probability                   0.4993 ( 49.93%)
Fake Probability                   0.5007 ( 50.07%)
----------------------------------------------------------------------
Predicted Label                           FAKE
Confidence                              50.07%
Threshold Used                            0.50

⚠️  WARNING: This video is predicted to be FAKE/DEEPFAKE!


📊 Result Summary:
   video_path: video.mp4
   predicted_label: FAKE
   prob_real: 0.499280959367

In [ ]:
# ============================================================
# RE-SAVE MODEL TO .keras FORMAT (Run this after training)
# ============================================================

print("Re-saving model to .keras format...")

# Load the .h5 model
from tensorflow.keras.models import load_model

# Define custom objects (copy from training code)
custom_objects = {
    'VisionTemporalTransformer': VisionTemporalTransformer,
    'consistency_loss_wrapper': consistency_loss_wrapper
}

# Load H5 model
model_h5 = load_model('lipinc_celebdf2_trained.h5',
                       custom_objects=custom_objects,
                       compile=False)

# Save in .keras format
model_h5.save('lipinc_celebdf2_trained.keras')
print("✅ Model saved to lipinc_celebdf2_trained.keras")

# Now you can load it without issues:
# model = load_model('lipinc_celebdf2_trained.keras', custom_objects=custom_objects)

Re-saving model to .keras format...


NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 128), dtype=float32, sparse=False, ragged=False, name=keras_tensor_141>',)
  • kwargs={'mask': 'None'}